# Import Libraries


In [28]:
import requests
import pandas as pd
import numpy as np
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.static import players
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonallplayers
from nba_api.stats.endpoints import commonplayerinfo
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.static import teams
from nba_api.stats.endpoints import playervsplayer
from nba_api.stats.endpoints import boxscoredefensive
from nba_api.stats.endpoints import boxscoreplayertrackv2
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import leaguedashptdefend
from nba_api.stats.endpoints import playbyplay
from nba_api.stats.endpoints import leagueseasonmatchups
from nba_api.stats.endpoints import teaminfocommon
from nba_api.stats.endpoints import playerdashboardbyshootingsplits
from nba_api.stats.endpoints import teamdashboardbygamesplits
from nba_api.stats.endpoints import teamestimatedmetrics



In [2]:
# Get all the teams and their information
all_teams = teams.get_teams()
all_teams_df = pd.DataFrame(all_teams)
abb_t = all_teams_df.loc[all_teams_df['full_name'] == 'Boston Celtics', 'abbreviation'].item()
abb_t
# Extract the team abbreviations
# team_abbreviations = [(team['full_name'],team['abbreviation']) for team in all_teams]
# team_abbreviations

# [('Atlanta Hawks', 'ATL'),('Boston Celtics', 'BOS'),('Cleveland Cavaliers', 'CLE'),('New Orleans Pelicans', 'NOP'),('Chicago Bulls', 'CHI'),('Dallas Mavericks', 'DAL'),
#  ('Denver Nuggets', 'DEN'),('Golden State Warriors', 'GSW'),('Houston Rockets', 'HOU'),('Los Angeles Clippers', 'LAC'),('Los Angeles Lakers', 'LAL'),('Miami Heat', 'MIA'),
#   ('Milwaukee Bucks', 'MIL'),('Minnesota Timberwolves', 'MIN'),('Brooklyn Nets', 'BKN'),('New York Knicks', 'NYK'),('Orlando Magic', 'ORL'),('Indiana Pacers', 'IND'),
#  ('Philadelphia 76ers', 'PHI'),('Phoenix Suns', 'PHX'),('Portland Trail Blazers', 'POR'),('Sacramento Kings', 'SAC'),('San Antonio Spurs', 'SAS'),('Oklahoma City Thunder', 'OKC'),
#  ('Toronto Raptors', 'TOR'),('Utah Jazz', 'UTA'),('Memphis Grizzlies', 'MEM'),('Washington Wizards', 'WAS'),('Detroit Pistons', 'DET'),('Charlotte Hornets', 'CHA')]

'BOS'

# Get Team's Players via Seasons and Team Abb

In [3]:
# Specify the team's abbreviation
team_abbreviation = "DEN" 
season = "2022-23"

## Get Team's ID via Team Abb

In [4]:
# Find the team by its abbreviation
team_info = teams.find_team_by_abbreviation(team_abbreviation)

# Retrieve the team ID
team_id = team_info['id']
team_id

1610612743

## Get Team's Roster & Player Names

In [5]:
top_n_players = 8

In [6]:
# Get the current season's roster for the team
roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
roster = roster.get_data_frames()[0]
roster.head()
# roster.head(20)

,TeamID,SEASON,LeagueID,PLAYER,NICKNAME,PLAYER_SLUG,NUM,POSITION,HEIGHT,WEIGHT,BIRTH_DATE,AGE,EXP,SCHOOL,PLAYER_ID,HOW_ACQUIRED
0,1610612743,2022,00,Christian Braun,Christian,christian-braun,0,G,6-6,220,"APR 17, 2001",22.0,R,Kansas,1631128,#21 Pick in 2022 Draft
1,1610612743,2022,00,Michael Porter Jr.,Michael,michael-porter-jr,1,F,6-10,218,"JUN 29, 1998",24.0,3,Missouri,1629008,#14 Pick in 2018 Draft
2,1610612743,2022,00,Kentavious Caldwell-Pope,Kentavious,kentavious-caldwell-pope,5,G,6-5,204,"FEB 18, 1993",30.0,9,Georgia,203484,Traded from WAS on 07/06/22
3,1610612743,2022,00,DeAndre Jordan,DeAndre,deandre-jordan,6,C,6-11,265,"JUL 21, 1988",34.0,14,Texas A&M,201599,Signed on 07/11/22
4,1610612743,2022,00,Reggie Jackson,Reggie,reggie-jackson,7,G,6-2,208,"APR 16, 1990",33.0,11,Boston College,202704,Signed on 02/14/23


In [7]:
player_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season, per_mode_detailed='Totals', last_n_games = 82, season_type_all_star = "Playoffs").get_data_frames()[0]
# player_stats.loc[player_stats['TEAM_ID'] == team_id, :]
player_mins = player_stats.loc[player_stats['TEAM_ID'] == team_id, ['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN']].sort_values('MIN', ascending=False)
top_n_mins = player_mins[:top_n_players]
top_n_mins_players = top_n_mins['PLAYER_NAME']

# top_n_mins

# Get Player ID via Name

In [8]:
# Specify the player's full name
player_name = "Jamal Murray"
player_name2 = "Lebron James"

# Get the player's ID using their name
player_info = players.find_players_by_full_name(player_name)
player_info2 = players.find_players_by_full_name(player_name2)
player_id = player_info[0]['id']
player_id2 = player_info2[0]['id']

# Get Player Box Score for Last N Games

In [9]:
Last_N = 10

In [10]:
# Get the playoff game log for the player
player_log = playergamelog.PlayerGameLog(player_id=player_id, season_type_all_star="Playoffs").get_data_frames()[0]
# Options:  ^(Regular Season)|(Pre Season)|(Playoffs)|(All-Star)|(All Star)$

# Retrieve the data from the player game log
player_log_data = player_log[:Last_N]

# Display the box score statistics
player_log_data.head()

,SEASON_ID,Player_ID,Game_ID,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,VIDEO_AVAILABLE
0,42022,1627750,0042200405,"JUN 12, 2023",DEN vs. MIA,W,41,6,15,0.400,...,8,8,8,1,0,6,2,14,12,1
1,42022,1627750,0042200404,"JUN 09, 2023",DEN @ MIA,W,43,5,17,0.294,...,3,3,12,1,0,0,1,15,22,1
2,42022,1627750,0042200403,"JUN 07, 2023",DEN @ MIA,W,45,12,22,0.545,...,7,10,10,1,0,7,2,34,14,1
3,42022,1627750,0042200402,"JUN 04, 2023",DEN vs. MIA,L,39,7,15,0.467,...,3,4,10,1,0,1,2,18,-2,1
4,42022,1627750,0042200401,"JUN 01, 2023",DEN vs. MIA,W,44,11,22,0.500,...,5,6,10,1,0,3,2,26,9,1


## Cleaning up the table & Introduce "Game Score"
Game Score = Points + 0.4 * Field Goals Made - 0.7 * Field Goals Attempted - 0.4 * (Free Throws Attempted - Free Throws Made) + 0.7 * Offensive Rebounds + 0.3 * Defensive Rebounds + Steals + 0.7 * Assists + 0.7 * Blocks - 0.4 * Personal Fouls - Turnovers

In [11]:
gs_col = list(player_log_data.columns)[3:-1]

player_log_gs = player_log_data.loc[:, gs_col]

player_log_gs["GS"] = player_log_gs["PTS"] + 0.4 * player_log_gs["FGM"] - 0.7 * player_log_gs["FGA"] - 0.4 * (player_log_gs["FTA"] - player_log_gs["FTM"])  + 0.7 * player_log_gs["OREB"] + 0.3 * player_log_gs["DREB"] + player_log_gs["STL"] + 0.7 * player_log_gs["AST"] + 0.7 * player_log_gs["BLK"] - 0.4 * player_log_gs["PF"] - player_log_gs["TOV"]
player_log_gs["GS_MIN"] = round(player_log_gs["GS"] / player_log_gs["MIN"], 3)
player_log_gs.head()

,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,GS,GS_MIN
0,"JUN 12, 2023",DEN vs. MIA,W,41,6,15,0.400,2,7,0.286,...,8,8,1,0,6,2,14,12,8.1,0.198
1,"JUN 09, 2023",DEN @ MIA,W,43,5,17,0.294,2,3,0.667,...,3,12,1,0,0,1,15,22,15.0,0.349
2,"JUN 07, 2023",DEN @ MIA,W,45,12,22,0.545,3,6,0.500,...,10,10,1,0,7,2,34,14,27.4,0.609
3,"JUN 04, 2023",DEN vs. MIA,L,39,7,15,0.467,3,8,0.375,...,4,10,1,0,1,2,18,-2,18.1,0.464
4,"JUN 01, 2023",DEN vs. MIA,W,44,11,22,0.500,2,7,0.286,...,6,10,1,0,3,2,26,9,21.4,0.486


## Average Game Score over Last N Games

### Split into Home, Away

In [12]:
def addCourt(df):
  if 'vs.' in df['MATCHUP']:
    df['COURT'] = 'HOME'
  else:
    df['COURT'] = 'AWAY'
  return df

In [13]:
player_log_gs = player_log_gs.apply(addCourt, axis=1)
player_log_gs

,GAME_DATE,MATCHUP,WL,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,...,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,GS,GS_MIN,COURT
0,"JUN 12, 2023",DEN vs. MIA,W,41,6,15,0.400,2,7,0.286,...,8,1,0,6,2,14,12,8.1,0.198,HOME
1,"JUN 09, 2023",DEN @ MIA,W,43,5,17,0.294,2,3,0.667,...,12,1,0,0,1,15,22,15.0,0.349,AWAY
2,"JUN 07, 2023",DEN @ MIA,W,45,12,22,0.545,3,6,0.500,...,10,1,0,7,2,34,14,27.4,0.609,AWAY
3,"JUN 04, 2023",DEN vs. MIA,L,39,7,15,0.467,3,8,0.375,...,10,1,0,1,2,18,-2,18.1,0.464,HOME
4,"JUN 01, 2023",DEN vs. MIA,W,44,11,22,0.500,2,7,0.286,...,10,1,0,3,2,26,9,21.4,0.486,HOME
5,"MAY 22, 2023",DEN @ LAL,W,43,10,18,0.556,0,4,0.000,...,5,2,1,2,3,25,11,20.3,0.472,AWAY
6,"MAY 20, 2023",DEN @ LAL,W,42,15,29,0.517,5,11,0.455,...,6,2,0,1,2,37,18,29.2,0.695,AWAY
7,"MAY 18, 2023",DEN vs. LAL,W,43,11,24,0.458,6,14,0.429,...,5,4,0,3,1,37,8,32.1,0.747,HOME
8,"MAY 16, 2023",DEN vs. LAL,W,37,12,20,0.600,4,8,0.500,...,5,3,1,1,5,31,3,27.9,0.754,HOME
9,"MAY 11, 2023",DEN @ PHX,W,35,7,16,0.438,4,7,0.571,...,4,4,0,3,1,26,6,22.2,0.634,AWAY


In [14]:
player_avg = player_log_gs.groupby('COURT').mean(numeric_only=True).round(2).reindex(['HOME', 'AWAY'])
player_avg.columns.name = player_name
player_avg

Jamal Murray,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,...,REB,AST,STL,BLK,TOV,PF,PTS,PLUS_MINUS,GS,GS_MIN
COURT,,,,,,,,,,,,,,,,,,,,,
HOME,40.8,9.4,19.2,0.48,3.4,8.8,0.38,3.0,3.2,0.78,...,6.6,7.6,2.0,0.2,2.8,2.4,25.2,6.0,21.52,0.53
AWAY,41.6,9.8,20.4,0.47,2.8,6.2,0.44,5.0,5.2,0.98,...,5.4,7.4,2.0,0.2,2.6,1.8,27.4,14.2,22.82,0.55


# Player Shot Breakdown

## Overall Shot Summary

In [15]:
player_log_gs['FG2A'] = player_log_gs.FGA - player_log_gs.FG3A
player_log_gs['FG2M'] = player_log_gs.FGM - player_log_gs.FG3M
player_log_gs['FG2_PCT'] = round(player_log_gs.FG2M / player_log_gs.FG2A, 3)
# player_log_gs
player_log_shot = player_log_gs.loc[:, ['FGM', 'FGA', 'FG_PCT', 'FG2M', 'FG2A', 'FG2_PCT', 'FG3M', 'FG3A', 'FG3_PCT']] 

player_log_shot.columns.name = player_name
player_log_shot['SHOT2_PCT'] = round(player_log_shot.FG2A / player_log_shot.FGA, 3)
player_log_shot['SHOT3_PCT'] = 1 -  player_log_shot.SHOT2_PCT

player_log_shot.mean().reset_index()

,Jamal Murray,0
0,FGM,9.6000
1,FGA,19.8000
2,FG_PCT,0.4775
3,FG2M,6.5000
4,FG2A,12.3000
5,FG2_PCT,0.5217
6,FG3M,3.1000
7,FG3A,7.5000
8,FG3_PCT,0.4069
9,SHOT2_PCT,0.6211


## Shot Location

In [16]:
pvp_log = playervsplayer.PlayerVsPlayer(last_n_games=7, measure_type_detailed_defense="Base", month=0, opponent_team_id=0, pace_adjust="N", 
                                        per_mode_detailed="PerGame", period=0, player_id=player_id, plus_minus="N", rank="N", season="2022-23", season_type_playoffs="Playoffs",
                                        vs_player_id= player_id2, vs_division_nullable=None, vs_conference_nullable=None, season_segment_nullable=None, outcome_nullable=None,
                                        location_nullable=None, game_segment_nullable	=None, date_to_nullable=None, date_from_nullable=None
                                        )

In [17]:
pvp_df = pvp_log.get_data_frames()

In [18]:
pvp_overall, pvp_vs = pvp_df[0], pvp_df[1]
pvp_shotDist, pvp_shotDistVS_on, pvp_shotDistVS_off = pvp_df[2], pvp_df[3], pvp_df[4]
pvp_shotType, pvp_shotTypeVS_on, pvp_shotTypeVS_off = pvp_df[5], pvp_df[6], pvp_df[7]

In [19]:
pvp_vs.loc[:, ['PLAYER_NAME',
       'VS_PLAYER_NAME', 'COURT_STATUS', 'GP', 'MIN', 'FGM', 
       'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA',
       'OREB', 'DREB', 'REB', 'AST', 'TOV', 'STL', 'BLK', 'BLKA',
       'PTS',]]

,PLAYER_NAME,VS_PLAYER_NAME,COURT_STATUS,GP,MIN,FGM,FGA,FG3M,FG3A,FTM,FTA,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PTS
0,Jamal Murray,"James, LeBron",On,2,39.8,12.5,23.0,2.5,7.0,3.5,3.5,0.0,4.5,4.5,5.0,1.5,1.5,0.5,0.5,31.0
1,Jamal Murray,"James, LeBron",Off,2,2.6,0.0,0.5,0.0,0.5,0.0,0.0,0.0,0.5,0.5,0.5,0.0,0.5,0.0,0.0,0.0


In [20]:
# Last 7 Games, where shot breakdown
pvp_shotType

,GROUP_SET,GROUP_VALUE,PLAYER_ID,PLAYER_NAME,FGM,FGA,FG_PCT
0,Shot Area,Restricted Area,1627750,Jamal Murray,2.6,3.9,0.667
1,Shot Area,In The Paint (Non-RA),1627750,Jamal Murray,1.7,3.7,0.462
2,Shot Area,Mid-Range,1627750,Jamal Murray,2.7,5.6,0.487
3,Shot Area,Left Corner 3,1627750,Jamal Murray,0.3,0.7,0.400
4,Shot Area,Right Corner 3,1627750,Jamal Murray,0.0,0.0,0.000
5,Shot Area,Above the Break 3,1627750,Jamal Murray,2.1,5.9,0.366
6,Shot Area,Backcourt,1627750,Jamal Murray,0.0,0.0,0.000


In [21]:
pvp_shotDist

,GROUP_SET,GROUP_VALUE,PLAYER_ID,PLAYER_NAME,FGM,FGA,FG_PCT
0,Shot Distance (5ft),Less Than 5 ft.,1627750,Jamal Murray,2.6,3.9,0.667
1,Shot Distance (5ft),5-9 ft.,1627750,Jamal Murray,1.1,2.1,0.533
2,Shot Distance (5ft),10-14 ft.,1627750,Jamal Murray,1.4,3.1,0.455
3,Shot Distance (5ft),15-19 ft.,1627750,Jamal Murray,1.3,2.9,0.450
4,Shot Distance (5ft),20-24 ft.,1627750,Jamal Murray,1.1,2.1,0.533
5,Shot Distance (5ft),25-29 ft.,1627750,Jamal Murray,1.9,5.0,0.371
6,Shot Distance (5ft),30-34 ft.,1627750,Jamal Murray,0.0,0.6,0.000
7,Shot Distance (5ft),35-39 ft.,1627750,Jamal Murray,0.0,0.0,0.000
8,Shot Distance (5ft),40+ ft.,1627750,Jamal Murray,0.0,0.0,0.000


## Overall Shooting Splits Test 2

In [22]:
playerShootingSplits = playerdashboardbyshootingsplits.PlayerDashboardByShootingSplits(last_n_games=7, measure_type_detailed="Base", month=0, opponent_team_id=0, pace_adjust="N", 
                                        per_mode_detailed="PerGame", period=0, player_id=player_id, plus_minus="N", rank="N", season="2022-23", season_type_playoffs="Playoffs",
                                        vs_division_nullable=None, vs_conference_nullable=None, season_segment_nullable=None, outcome_nullable=None,
                                        location_nullable=None, game_segment_nullable	=None, date_to_nullable=None, date_from_nullable=None
                                        )

In [23]:
playerShootingSplits.shot_area_player_dashboard.get_data_frame()

,GROUP_SET,GROUP_VALUE,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,EFG_PCT,BLKA,...,FG3A_RANK,FG3_PCT_RANK,EFG_PCT_RANK,BLKA_RANK,PCT_AST_2PM_RANK,PCT_UAST_2PM_RANK,PCT_AST_3PM_RANK,PCT_UAST_3PM_RANK,PCT_AST_FGM_RANK,PCT_UAST_FGM_RANK
0,Shot Area,Restricted Area,18,27,0.667,0,0,0.000,0.667,3,...,3,3,3,6,1,2,3,2,2,2
1,Shot Area,In The Paint (Non-RA),12,26,0.462,0,0,0.000,0.462,4,...,3,3,4,7,2,2,3,2,3,2
2,Shot Area,Mid-Range,19,39,0.487,0,0,0.000,0.487,1,...,3,3,2,4,2,1,3,2,3,1
3,Shot Area,Left Corner 3,2,5,0.400,2,5,0.400,0.600,1,...,2,2,5,4,4,4,2,2,5,5
4,Shot Area,Right Corner 3,0,0,0.000,0,0,0.000,0.000,0,...,3,3,6,1,4,4,3,2,6,5
5,Shot Area,Above the Break 3,15,41,0.366,15,41,0.366,0.549,0,...,1,1,1,1,4,4,1,1,1,4
6,Shot Area,Backcourt,0,0,0.000,0,0,0.000,0.000,0,...,3,3,6,1,4,4,3,2,6,5


# Get Matchup Stats (Work In Progress)


In [24]:
# pvp_game = playbyplay.PlayByPlay(game_id = "0042200401").get_data_frames()[0]
# pvp_game.head()



In [25]:
# pvp_game.head()


,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_FG3A,MATCHUP_FG3_PCT,HELP_BLK,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL,MATCHUP_TIME_SEC
5444,22022,2544,LeBron James,201587,Nicolas Batum,4,5:08,35.4,24,51,...,7,0.429,0,0,0,0,1,2,2,307.5
5208,22022,2544,LeBron James,1629611,Terance Mann,4,6:45,36.1,9,28,...,4,0.250,0,0,0,0,0,0,0,405.4
8042,22022,2544,LeBron James,1627826,Ivica Zubac,4,5:01,29.2,22,38,...,9,0.667,0,0,0,0,0,0,0,301.2
28950,22022,2544,LeBron James,1626181,Norman Powell,4,2:20,12.6,3,16,...,1,1.000,0,0,0,0,0,0,0,139.7
12207,22022,2544,LeBron James,1628971,Bruce Brown,3,4:01,23.1,10,34,...,3,0.000,0,0,0,0,2,2,1,240.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65017,22022,2544,LeBron James,1629644,KZ Okpala,1,1:14,5.2,4,8,...,0,0.000,0,0,0,0,0,0,0,73.8
66223,22022,2544,LeBron James,1629669,Jaylen Nowell,1,0:49,5.0,4,4,...,0,0.000,0,0,0,0,2,2,1,49.0
66825,22022,2544,LeBron James,1630256,Jae'Sean Tate,1,0:48,4.9,7,9,...,2,0.500,0,0,0,0,2,2,1,47.8
66855,22022,2544,LeBron James,1630165,Killian Hayes,1,0:38,4.9,4,5,...,0,0.000,0,0,0,0,0,0,0,38.2


,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,HELP_BLK,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL
67609,22022,2544,LeBron James,1626181,Norman Powell,4,0.6,3.2,0.8,4.0,...,0.3,0.3,1.000,0,0,0,0,0.0,0.0,0.0
27261,22022,2544,LeBron James,1627826,Ivica Zubac,4,1.3,7.3,5.5,9.5,...,1.5,2.3,0.667,0,0,0,0,0.0,0.0,0.0
20212,22022,2544,LeBron James,201587,Nicolas Batum,4,1.3,8.9,6.0,12.8,...,0.8,1.8,0.429,0,0,0,0,0.3,0.5,0.5
19864,22022,2544,LeBron James,1629611,Terance Mann,4,1.7,9.0,2.3,7.0,...,0.3,1.0,0.250,0,0,0,0,0.0,0.0,0.0
41832,22022,2544,LeBron James,203468,CJ McCollum,3,1.0,5.2,4.0,10.3,...,0.0,0.0,0.000,0,0,0,0,0.7,1.3,0.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44562,22022,2544,LeBron James,1629669,Jaylen Nowell,1,0.8,5.0,4.0,4.0,...,0.0,0.0,0.000,0,0,0,0,2.0,2.0,1.0
44801,22022,2544,LeBron James,201939,Stephen Curry,1,0.8,4.9,0.0,6.0,...,0.0,0.0,0.000,0,0,0,0,0.0,0.0,0.0
44957,22022,2544,LeBron James,1629013,Landry Shamet,1,0.9,4.9,0.0,4.0,...,0.0,0.0,0.000,0,0,0,0,0.0,0.0,0.0
45036,22022,2544,LeBron James,1630165,Killian Hayes,1,0.6,4.9,4.0,5.0,...,0.0,0.0,0.000,0,0,0,0,0.0,0.0,0.0


## Things to analyse
1. Get the Offensive player use all the past 5 seasons > then generate all the possible defensive players
2. Draw a graph for a possible statistic to show whether someone has been defending better. (potentially pts)
3. Compare between with him on defense vs his average stats to see if he is a good defensive player comparably to average

In [31]:
defend = leaguedashptdefend.LeagueDashPtDefend(defense_category = 'Overall', league_id='00', per_mode_simple = 'Totals')
defend.get_data_frames()[0].head()

,CLOSE_DEF_PERSON_ID,PLAYER_NAME,PLAYER_LAST_TEAM_ID,PLAYER_LAST_TEAM_ABBREVIATION,PLAYER_POSITION,AGE,GP,G,FREQ,D_FGM,D_FGA,D_FG_PCT,NORMAL_FG_PCT,PCT_PLUSMINUS
0,201572,Brook Lopez,1610612749,MIL,C,35.0,78,78,1.0,810,1761,0.460,0.504,-0.044
1,1627734,Domantas Sabonis,1610612758,SAC,F-C,27.0,79,79,1.0,782,1453,0.538,0.511,0.028
2,1630596,Evan Mobley,1610612739,CLE,C,21.0,79,79,1.0,666,1416,0.470,0.487,-0.017
3,203999,Nikola Jokic,1610612743,DEN,C,28.0,69,69,1.0,712,1399,0.509,0.501,0.008
4,202696,Nikola Vucevic,1610612741,CHI,C,32.0,82,82,1.0,735,1392,0.528,0.508,0.020


In [38]:
pvp_game = leagueseasonmatchups.LeagueSeasonMatchups(league_id = "00", 	per_mode_simple = "PerGame", season = season, season_type_playoffs = "Regular Season").get_data_frames()[0]
# pvp_game.head()
pvp_df = pvp_game.loc[pvp_game['OFF_PLAYER_ID'] == player_id, :]
pvp_df = pvp_df.loc[pvp_df['DEF_PLAYER_ID'] == player_id2, :]
pvp_df

,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,HELP_BLK,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL
69682,22022,1627750,Jamal Murray,2544,LeBron James,3,0.6,3.0,2.3,4.7,...,0.3,1.0,0.333,0,0,0,0,0.0,0.0,0.0


In [42]:
cat_pvp = []
season_lst = ['2022-23', '2021-22', '2020-21', '2019-20', '2018-19', '2017-18', '2008-09']
for s in season_lst:
    try:
        print("Processing season = {}".format(s))
        pvp_s = leagueseasonmatchups.LeagueSeasonMatchups(league_id = "00", per_mode_simple = "PerGame", season = s, season_type_playoffs = "Regular Season", off_player_id_nullable = player_id, def_player_id_nullable = player_id2).get_data_frames()[0]
        print("Appending Season = {} to the list".format(s))
        cat_pvp.append(pvp_s)
    except Exception as e:
        print("Season = {}, there was an exception".format(s))


cat_s = pd.concat(cat_pvp)
cat_s

Processing season = 2022-23
Appending Season = 2022-23 to the list
Processing season = 2021-22
Appending Season = 2021-22 to the list
Processing season = 2020-21
Appending Season = 2020-21 to the list
Processing season = 2019-20
Appending Season = 2019-20 to the list
Processing season = 2018-19
Appending Season = 2018-19 to the list
Processing season = 2017-18
Appending Season = 2017-18 to the list
Processing season = 2008-09
Appending Season = 2008-09 to the list


,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_FG3M,MATCHUP_FG3A,MATCHUP_FG3_PCT,HELP_BLK,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL
0,22022,1627750,Jamal Murray,2544,LeBron James,3,0.6,3.0,2.3,4.7,...,0.3,1.0,0.333,0,0,0,0,0.0,0.0,0.0
0,22020,1627750,Jamal Murray,2544,LeBron James,2,0.9,5.0,2.0,9.0,...,0.0,0.5,0.000,0,0,0,0,0.0,0.0,0.0
0,22019,1627750,Jamal Murray,2544,LeBron James,3,0.5,3.1,1.7,5.0,...,0.0,0.0,0.000,0,0,0,0,0.3,0.7,0.3
0,22018,1627750,Jamal Murray,2544,LeBron James,3,1.0,4.8,0.7,8.0,...,0.0,0.7,0.000,0,0,0,0,0.0,0.0,0.0
0,22017,1627750,Jamal Murray,2544,LeBron James,2,0.9,3.6,0.0,5.5,...,0.0,0.5,0.000,0,0,0,0,0.0,0.0,0.0


In [32]:
pvp_game = leagueseasonmatchups.LeagueSeasonMatchups(league_id = "00", 	per_mode_simple = "Totals", season = season, season_type_playoffs = "Regular Season").get_data_frames()[0]
pvp_game.loc[pvp_game['OFF_PLAYER_ID'] == player_id2, :].sort_values('GP', ascending=False)

,SEASON_ID,OFF_PLAYER_ID,OFF_PLAYER_NAME,DEF_PLAYER_ID,DEF_PLAYER_NAME,GP,MATCHUP_MIN,PARTIAL_POSS,PLAYER_PTS,TEAM_PTS,...,MATCHUP_FG3A,MATCHUP_FG3_PCT,HELP_BLK,HELP_FGM,HELP_FGA,HELP_FG_PERC,MATCHUP_FTM,MATCHUP_FTA,SFL,MATCHUP_TIME_SEC
5443,22022,2544,LeBron James,201587,Nicolas Batum,4,5:08,35.4,24,51,...,7,0.429,0,0,0,0,1,2,2,307.5
5195,22022,2544,LeBron James,1629611,Terance Mann,4,6:45,36.1,9,28,...,4,0.250,0,0,0,0,0,0,0,405.4
28847,22022,2544,LeBron James,1626181,Norman Powell,4,2:20,12.6,3,16,...,1,1.000,0,0,0,0,0,0,0,139.7
8053,22022,2544,LeBron James,1627826,Ivica Zubac,4,5:01,29.2,22,38,...,9,0.667,0,0,0,0,0,0,0,301.2
12151,22022,2544,LeBron James,1628971,Bruce Brown,3,4:01,23.1,10,34,...,3,0.000,0,0,0,0,2,2,1,240.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64917,22022,2544,LeBron James,1629644,KZ Okpala,1,1:14,5.2,4,8,...,0,0.000,0,0,0,0,0,0,0,73.8
66345,22022,2544,LeBron James,1629669,Jaylen Nowell,1,0:49,5.0,4,4,...,0,0.000,0,0,0,0,2,2,1,49.0
67164,22022,2544,LeBron James,1630256,Jae'Sean Tate,1,0:48,4.9,7,9,...,2,0.500,0,0,0,0,2,2,1,47.8
67202,22022,2544,LeBron James,1628973,Jalen Brunson,1,0:48,4.9,0,7,...,0,0.000,0,0,0,0,0,0,0,48.4


# Top Impact Players in Team via GS

## GS Summary Function

In [ ]:
from requests.models import ReadTimeoutError
def getRosterImpact(team_abb, game_type = "Regular Season", last_n_games = 5):
  def p_addCourt(df):
    if 'vs.' in df['MATCHUP']:
      df['COURT'] = 'HOME'
    else:
      df['COURT'] = 'AWAY'
    return df

  # Find the team by its abbreviation
  t_info = teams.find_team_by_abbreviation(team_abb)
  # Retrieve the team ID
  t_id = t_info['id']
  t_id

  top_n = 8
  p_stats = leaguedashplayerstats.LeagueDashPlayerStats(season=season, per_mode_detailed='Totals', last_n_games = 82, season_type_all_star = "Playoffs").get_data_frames()[0]
  # player_stats.loc[player_stats['TEAM_ID'] == team_id, :]
  p_mins = p_stats.loc[p_stats['TEAM_ID'] == t_id, ['PLAYER_NAME', 'TEAM_ABBREVIATION', 'GP', 'MIN']].sort_values('MIN', ascending=False)
  top_mins = p_mins[:top_n]
  top_mins_players = top_mins['PLAYER_NAME']

  dfs = []
  use_cols = ['GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']
  # p = top_mins_players[0]
  for p in top_mins_players:
    print("Processing Player : {}".format(p))
    p_id = players.find_players_by_full_name(p)[0]['id']
    # print("Player : {} \t Player_ID = {}".format(p, player_id))
    try:
      p_log = playergamelog.PlayerGameLog(player_id=p_id, season_type_all_star=game_type, timeout=10).get_data_frames()[0][:last_n_games]
      if len(p_log) > 0:
        p_log_gs = p_log.loc[:, use_cols[1:-1]]
        p_log_gs["GS"] = p_log_gs["PTS"] + 0.4 * p_log_gs["FGM"] - 0.7 * p_log_gs["FGA"] - 0.4 * (p_log_gs["FTA"] - p_log_gs["FTM"])  + 0.7 * p_log_gs["OREB"] + 0.3 * p_log_gs["DREB"] + p_log_gs["STL"] + 0.7 * p_log_gs["AST"] + 0.7 * p_log_gs["BLK"] - 0.4 * p_log_gs["PF"] - p_log_gs["TOV"]
        p_log_gs["GS_MIN"] = round(p_log_gs["GS"] / p_log_gs["MIN"], 3)
        p_log_gs= p_log_gs.apply(p_addCourt, axis=1)
        # print(p_log_gs)
        p_avg = p_log_gs.groupby('COURT').mean(numeric_only=True).round(2).reindex(['HOME', 'AWAY'])
        p_avg.columns.name = p
        dfs.append(p_avg)
    except requests.exceptions.ReadTimeout:
      print("Timeout Occured for Player: {}".format(p))
      continue
  df = pd.concat(dfs, axis=0, keys=top_mins_players)
  df.columns.name = "{} Games Avg".format(last_n_games)
  return df

## GS Summary Results

In [ ]:
getRosterImpact('DEN', game_type="Playoffs", last_n_games = 16)

Processing Player : Jamal Murray
Processing Player : Nikola Jokic
Processing Player : Aaron Gordon
Processing Player : Kentavious Caldwell-Pope
Processing Player : Michael Porter Jr.
Processing Player : Bruce Brown
Processing Player : Jeff Green
Processing Player : Christian Braun


16 Games Avg                      MIN    FGM    FGA  FG_PCT  FG3M  FG3A  \
PLAYER_NAME              COURT                                            
Jamal Murray             HOME   39.78   9.11  19.22    0.46  3.33  8.44   
                         AWAY   41.57  10.71  22.29    0.47  2.29  5.71   
Nikola Jokic             HOME   38.89  11.22  21.22    0.56  1.67  3.89   
                         AWAY   40.43  12.00  21.43    0.56  1.71  3.86   
Aaron Gordon             HOME   37.12   5.38   9.75    0.56  0.88  2.50   
                         AWAY   37.75   5.50  10.75    0.49  1.00  2.25   
Kentavious Caldwell-Pope HOME   32.00   3.50   7.88    0.43  2.00  4.75   
                         AWAY   35.12   3.50   7.25    0.44  1.50  4.38   
Michael Porter Jr.       HOME   32.00   4.38  10.38    0.42  2.25  6.38   
                         AWAY   33.00   4.38  11.62    0.37  2.12  6.75   
Bruce Brown              HOME   27.00   4.75   8.88    0.52  1.00  2.38   
                         AWAY   26.25   4.25   8.50    0.55  1.00  3.12   
Jeff Green               HOME   16.12   1.12   3.00    0.41  0.38  1.25   
                         AWAY   17.00   1.12   2.12    0.47  0.50  1.25   
Christian Braun          HOME   12.50   0.88   2.25    0.40  0.12  0.62   
                         AWAY   12.00   1.62   2.50    0.33  0.12  0.62   

16 Games Avg                    FG3_PCT   FTM   FTA  FT_PCT  ...   DREB  \
PLAYER_NAME              COURT                               ...          
Jamal Murray             HOME      0.39  3.33  3.44    0.88  ...   4.89   
                         AWAY      0.38  4.43  4.86    0.89  ...   4.43   
Nikola Jokic             HOME      0.46  5.78  7.00    0.81  ...  11.22   
                         AWAY      0.40  6.29  7.43    0.86  ...   9.29   
Aaron Gordon             HOME      0.32  2.50  3.88    0.55  ...   3.00   
                         AWAY      0.27  1.62  2.88    0.53  ...   3.50   
Kentavious Caldwell-Pope HOME      0.42  1.38  1.75    0.44  ...   2.38   
                         AWAY      0.26  1.62  1.88    0.50  ...   2.75   
Michael Porter Jr.       HOME      0.32  0.62  0.62    0.38  ...   6.75   
                         AWAY      0.23  1.50  1.88    0.72  ...   6.62   
Bruce Brown              HOME      0.38  3.38  3.62    0.59  ...   3.88   
                         AWAY      0.25  1.38  1.62    0.65  ...   3.00   
Jeff Green               HOME      0.31  1.00  1.25    0.25  ...   1.38   
                         AWAY      0.29  1.00  1.00    0.38  ...   1.25   
Christian Braun          HOME      0.12  0.25  0.50    0.12  ...   1.38   
                         AWAY      0.06  0.62  0.88    0.38  ...   1.25   

16 Games Avg                      REB    AST   STL   BLK   TOV    PF    PTS  \
PLAYER_NAME              COURT                                                
Jamal Murray             HOME    5.67   7.33  1.56  0.11  2.67  1.89  24.89   
                         AWAY    5.43   7.00  1.71  0.29  2.14  2.00  28.14   
Nikola Jokic             HOME   15.44   9.11  1.00  1.00  3.56  2.56  29.89   
                         AWAY   12.00  10.71  1.14  1.57  3.57  4.00  32.00   
Aaron Gordon             HOME    4.88   2.12  0.50  0.75  1.12  2.00  14.12   
                         AWAY    6.50   3.75  0.38  0.75  0.88  3.25  13.62   
Kentavious Caldwell-Pope HOME    3.38   1.88  1.25  0.50  0.88  3.25  10.38   
                         AWAY    3.12   1.25  1.50  0.62  0.50  1.88  10.12   
Michael Porter Jr.       HOME    7.62   1.38  0.62  0.62  0.62  1.88  11.62   
                         AWAY    7.75   1.62  0.25  0.62  0.38  2.38  12.38   
Bruce Brown              HOME    4.38   1.75  1.12  0.25  1.38  1.75  13.88   
                         AWAY    3.75   1.62  1.00  0.62  0.88  3.25  10.88   
Jeff Green               HOME    2.00   0.50  0.12  0.38  0.38  2.12   3.62   
                         AWAY    1.38   0.50  0.25  0.25  0.38  2.00   3.75   
Christian Braun          HOME    2

In [ ]:
getRosterImpact('MIA', game_type="Playoffs", last_n_games = 16).sort_values('GS_MIN', ascending = False)

Processing Player : Jimmy Butler
Processing Player : Bam Adebayo
Processing Player : Caleb Martin
Processing Player : Gabe Vincent
Processing Player : Max Strus
Processing Player : Kyle Lowry
Processing Player : Duncan Robinson
Processing Player : Kevin Love


16 Games Avg             MIN   FGM    FGA  FG_PCT  FG3M  FG3A  FG3_PCT   FTM  \
PLAYER_NAME     COURT                                                          
Jimmy Butler    HOME   40.12  8.00  19.50    0.41  0.75  3.00     0.21  8.38   
                AWAY   38.88  7.62  18.12    0.41  1.00  2.62     0.23  4.38   
Bam Adebayo     AWAY   36.25  8.00  15.00    0.54  0.00  0.25     0.00  3.12   
Kyle Lowry      HOME   27.38  3.38   7.00    0.49  1.12  3.25     0.34  2.38   
Kevin Love      HOME   16.88  2.12   4.75    0.51  1.62  3.62     0.54  0.38   
Caleb Martin    AWAY   31.38  5.88  10.88    0.49  2.50  5.12     0.49  0.62   
Bam Adebayo     HOME   38.12  6.62  14.88    0.49  0.00  0.00     0.00  4.12   
Gabe Vincent    AWAY   33.62  5.12  11.62    0.45  3.00  6.88     0.43  2.12   
Duncan Robinson AWAY   19.62  3.88   7.50    0.47  2.25  5.00     0.45  0.25   
Caleb Martin    HOME   31.12  4.50   9.00    0.45  2.00  4.75     0.39  1.12   
Kevin Love      AWAY   19.38  2.50   7.50    0.34  1.62  5.12     0.26  0.38   
Max Strus       HOME   28.75  3.50   9.38    0.33  1.75  6.38     0.25  1.38   
                AWAY   26.75  3.62   9.00    0.39  2.38  7.00     0.35  0.62   
Kyle Lowry      AWAY   26.38  2.88   7.50    0.38  1.50  4.12     0.38  0.50   
Duncan Robinson HOME   16.50  3.12   7.62    0.39  2.12  5.75     0.36  0.50   
Gabe Vincent    HOME   29.75  3.75  10.12    0.33  1.62  5.38     0.25  1.50   

16 Games Avg            FTA  FT_PCT  ...  DREB    REB   AST   STL   BLK   TOV  \
PLAYER_NAME     COURT                ...                                        
Jimmy Butler    HOME   9.88    0.84  ...  4.38   6.88  5.88  0.88  0.88  1.75   
                AWAY   5.38    0.69  ...  4.00   5.75  6.75  2.75  0.62  1.50   
Bam Adebayo     AWAY   3.50    0.68  ...  7.88  10.38  4.50  0.62  0.62  2.62   
Kyle Lowry      HOME   2.50    0.60  ...  3.00   4.12  5.25  1.38  0.38  1.25   
Kevin Love      HOME   0.38    0.25  ...  4.12   4.50  1.12  0.25  0.25  0.25   
Caleb Martin    AWAY   0.62    0.38  ...  3.75   5.38  1.38  0.88  0.50  1.00   
Bam Adebayo     HOME   5.25    0.78  ...  7.12  10.38  2.00  0.75  0.75  2.62   
Gabe Vincent    AWAY   2.12    0.62  ...  1.62   2.00  3.50  0.88  0.25  1.12   
Duncan Robinson AWAY   0.38    0.12  ...  1.00   1.12  2.00  0.62  0.12  0.62   
Caleb Martin    HOME   1.38    0.44  ...  4.62   5.62  1.62  0.62  0.38  1.00   
Kevin Love      AWAY   0.62    0.19  ...  5.62   6.88  1.38  0.50  0.12  0.88   
Max Strus       HOME   1.62    0.44  ...  3.75   4.38  1.88  0.50  0.38  0.25   
                AWAY   0.88    0.18  ...  2.12   2.50  1.38  0.00  0.12  0.00   
Kyle Lowry      AWAY   0.50    0.25  ...  2.88   2.88  4.12  0.75  0.50  2.12   
Duncan Robinson HOME   0.50    0.25  ...  1.00   1.00  1.50  0.25  0.00  0.88   
Gabe Vincent    HOME   1.75    0.44  ...  0.88   1.25  2.75  1.00  0.25  1.25   

16 Games Avg             PF    PTS     GS  GS_MIN  
PLAYER_NAME     COURT                              
Jimmy Butler    HOME   1.50  25.12  20.39    0.51  
                AWAY   1.75  20.62  18.73    0.47  
Bam Adebayo     AWAY   3.50  19.12  15.98    0.46  
Kyle Lowry      HOME   2.75  10.25  11.30    0.42  
Kevin Love      HOME   2.00   6.25   5.44    0.37  
Caleb Martin    AWAY   2.12  14.88  12.21    0.36  
Bam Adebayo     HOME   2.38  17.38  12.68    0.34  
Gabe Vincent    AWAY   2.75  15.38  11.31    0.33  
Duncan Robinson AWAY   1.88  10.25   7.62    0.33  
Caleb Martin    HOME   2.88  12.12   9.49    0.28  
Kevin Love      AWAY   2.00   7.00   5.09    0.25  
Max Strus       HOME   1.88  10.12   7.50    0.24  
                AWAY   2.00  10.25   6.45    0.23  
Kyle Lowry      AWAY   2.12   7.75   5.52    0.20  
Duncan Robinson HOME   1.88   8.88   4.76    0.20  
Gabe Vincent    HOME   3.50  10.62   5.91    0.17  

[16 rows x 21 columns]

# Betting First 4 mins using GS_MIN (WIP)

In [ ]:
# Starting Lineup

In [ ]:
# Obtaining Period 1 / First 4 Mins GS_MIN Impact

What should be taken into consideration? Overall playoffs form (against different teams) or games against the same team? 

In [ ]:
# Compare HOME / AWAY GS_MIN Impact to see the odds

# Team vs Team Standings

In [17]:
team_abbs = ['DEN', 'LAL']
team_ids = []
for i in team_abbs:
    team_info = teams.find_team_by_abbreviation(i)
    # Retrieve the team ID
    team_id1 = team_info['id']
    team_ids.append(team_id1)
print(team_ids)

teamSplits = teamdashboardbygamesplits.TeamDashboardByGameSplits(last_n_games=7, measure_type_detailed_defense="Base", month=0, opponent_team_id=team_ids[0], team_id=team_ids[1], pace_adjust="N", 
                                        per_mode_detailed="Totals", period=0, plus_minus="N", rank="N", season="2021-22", season_type_all_star="Regular Season",
                                        vs_division_nullable=None, vs_conference_nullable=None, season_segment_nullable=None, outcome_nullable=None,
                                        location_nullable=None, game_segment_nullable=None, date_to_nullable=None, date_from_nullable=None
                                        )

[1610612743, 1610612747]


In [26]:
teamSplits.get_data_frames()[4]

,GROUP_SET,GROUP_VALUE,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,TOV,STL,BLK,BLKA,PF,PFD,PTS,PLUS_MINUS,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,FGM_RANK,FGA_RANK,FG_PCT_RANK,FG3M_RANK,FG3A_RANK,FG3_PCT_RANK,FTM_RANK,FTA_RANK,FT_PCT_RANK,OREB_RANK,DREB_RANK,REB_RANK,AST_RANK,TOV_RANK,STL_RANK,BLK_RANK,BLKA_RANK,PF_RANK,PFD_RANK,PTS_RANK,PLUS_MINUS_RANK
0,By Actual Margin,Behind 16 - 20 Points,1,1,0,1.0,0.433333,1,1,1.000,1,1,1.000,2,2,1.000,0,0,0,0,0.0,0,0,0,0,1,5,5.0,7,1,1,1,7,7,7,1,6,7,1,7,7,1,7,7,7,7,1,4,6,1,1,7,7,4
1,By Actual Margin,Behind 11 - 15 Points,2,1,1,0.5,10.665000,10,20,0.500,5,12,0.417,5,6,0.833,1,7,8,8,3.0,0,0,0,3,6,30,6.0,1,1,2,2,4,3,4,5,2,2,4,3,4,4,4,4,5,3,3,4,6,1,2,3,4,3
2,By Actual Margin,Behind 6 - 10 Points,2,1,1,0.5,14.150000,8,26,0.308,2,6,0.333,17,18,0.944,4,10,14,6,5.0,1,2,4,12,13,35,-7.0,1,1,2,2,3,5,3,7,4,4,5,2,2,3,2,2,2,4,6,2,2,6,6,2,3,5
3,By Actual Margin,Behind 1 - 5 Points,2,1,1,0.5,14.491667,20,32,0.625,7,10,0.700,5,9,0.556,1,8,9,12,3.0,1,1,3,4,6,52,27.0,1,1,2,2,2,2,2,2,1,3,2,3,3,7,4,3,3,2,3,2,3,5,4,3,2,1
4,By Actual Margin,Tied,2,1,1,0.5,7.411667,9,16,0.563,2,3,0.667,3,3,1.000,1,7,8,5,1.0,0,1,0,3,2,23,14.0,1,1,2,2,6,4,5,3,4,5,3,5,6,1,4,4,5,5,2,4,3,1,2,6,5,2
5,By Actual Margin,Ahead 1 - 5 Points,2,1,1,0.5,45.703333,43,85,0.506,5,28,0.179,21,26,0.808,10,32,42,25,10.0,6,6,5,22,21,112,-20.0,1,1,2,2,1,1,1,4,2,1,6,1,1,5,1,1,1,1,7,1,1,7,7,1,1,6
6,By Actual Margin,Ahead 6 - 10 Points,2,1,1,0.5,8.145000,2,6,0.333,0,3,0.000,3,4,0.750,2,7,9,1,4.0,0,1,0,5,3,7,-31.0,1,1,2,2,5,6,6,6,7,5,7,5,5,6,3,4,3,6,5,4,3,1,5,5,6,7


In [29]:
teamMetrics = teamestimatedmetrics.TeamEstimatedMetrics(league_id='00', season='2022-23', season_type='Regular Season')

In [30]:
teamMetrics.get_data_frames()[0]

,TEAM_NAME,TEAM_ID,GP,W,L,W_PCT,MIN,E_OFF_RATING,E_DEF_RATING,E_NET_RATING,E_PACE,E_AST_RATIO,E_OREB_PCT,E_DREB_PCT,E_REB_PCT,E_TM_TOV_PCT,GP_RANK,W_RANK,L_RANK,W_PCT_RANK,MIN_RANK,E_OFF_RATING_RANK,E_DEF_RATING_RANK,E_NET_RATING_RANK,E_AST_RATIO_RANK,E_OREB_PCT_RANK,E_DREB_PCT_RANK,E_REB_PCT_RANK,E_TM_TOV_PCT_RANK,E_PACE_RANK
0,Golden State Warriors,1610612744,82,44,38,0.537,3966.0,113.5,111.4,2.1,104.2,20.4,0.244,0.760,0.507,0.155,1,11,11,11,12,9,12,10,2,14,16,10,29,1
1,San Antonio Spurs,1610612759,82,22,60,0.268,3971.0,107.2,117.7,-10.5,104.1,18.7,0.256,0.749,0.493,0.145,1,28,28,28,8,29,30,30,8,7,24,21,24,2
2,Indiana Pacers,1610612754,82,35,47,0.427,3951.0,111.0,115.4,-4.4,103.8,18.9,0.234,0.722,0.478,0.142,1,23,23,23,25,24,26,26,7,18,30,28,20,3
3,Memphis Grizzlies,1610612763,82,51,31,0.622,3956.0,112.3,108.3,4.0,103.7,18.2,0.265,0.759,0.512,0.131,1,5,5,5,22,14,2,4,15,6,18,6,6,4
4,Oklahoma City Thunder,1610612760,82,40,42,0.488,3971.0,112.3,111.4,0.8,103.6,17.2,0.247,0.729,0.484,0.125,1,19,19,19,8,15,13,14,26,13,29,27,4,5
5,Los Angeles Lakers,1610612747,82,43,39,0.524,3976.0,111.8,111.5,0.3,103.6,17.9,0.228,0.763,0.505,0.134,1,14,14,14,5,19,16,16,18,20,13,12,11,5
6,Minnesota Timberwolves,1610612750,82,42,40,0.512,3966.0,111.3,111.1,0.2,103.4,18.6,0.215,0.743,0.484,0.147,1,15,15,15,12,23,11,20,10,27,26,26,25,7
7,Charlotte Hornets,1610612766,82,27,55,0.329,3966.0,106.7,112.6,-5.9,103.3,17.8,0.238,0.755,0.491,0.137,1,27,27,27,12,30,20,27,20,16,22,23,14,8
8,Atlanta Hawks,1610612737,82,41,41,0.500,3971.0,113.9,113.6,0.3,103.1,17.6,0.251,0.758,0.502,0.124,1,17,17,17,8,7,22,16,22,8,19,15,3,9
9,Utah Jazz,1610612762,82,37,45,0.451,3961.0,112.7,113.8,-1.1,103.1,18.2,0.268,0.752,0.514,0.148,1,22,22,22,18,12,23,22,14,4,23,4,28,9
